## 6.1案例分析：重构‘策略’模式
### 6.1.1经典的‘策略’模式

In [6]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price*self.quantity
    
class Order:
    
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total:{:.2f} due {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC):
    
    @abstractmethod
    def discount(self, order):
        """返回折扣金额"""
        
class FidelityPromotion(Promotion):
    
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromotion(Promotion):
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

class LargeOrderPromotion(Promotion):
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [7]:
joe = Customer('joe', 0)
ann = Customer('ann', 1100)
cart = {LineItem('banana', 4, 0.5),
       LineItem('apple', 10, 1.5),
       LineItem('watermelon', 5, 5)}
Order(joe, cart, FidelityPromotion())

<Order total:42.00 due 42.00>

In [8]:
Order(ann, cart, FidelityPromotion())

<Order total:42.00 due 39.90>

### 6.1.2 使用函数实现‘策略’模式

In [15]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price*self.quantity
    
class Order:
    
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total:{:.2f} due {:.2f}>'
        return fmt.format(self.total(), self.due())
    
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [16]:
joe = Customer('joe', 0)
ann = Customer('ann', 1100)
cart = {LineItem('banana', 4, 0.5),
       LineItem('apple', 10, 1.5),
       LineItem('watermelon', 5, 5)}
Order(joe, cart, fidelity_promo)

<Order total:42.00 due 42.00>

In [17]:
Order(ann, cart, fidelity_promo)

<Order total:42.00 due 39.90>

### 6.1.3 选择最佳策略：简单的方式

In [18]:
promos = [fidelity_promo,  bulk_item_promo, large_order_promo]

def best_promo(order):
    return max(promo(order) for promo in promos)

Order(ann, cart, best_promo)

<Order total:42.00 due 39.90>

### 6.1.4找出模块中的全部策略

In [19]:
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']
def best_promo(order):
    return max(promo(order) for promo in promos)

Order(ann, cart, best_promo)

<Order total:42.00 due 39.90>

In [21]:
import inspect
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]
def best_promo(order):
    return max(promo(order) for promo in promos)

Order(ann, cart, best_promo)

NameError: name 'promotions' is not defined